In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import torch
import torch.nn as nn 
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import random

from crossView_UCLA_ske import NUCLA_CrossView
from Dyan import Dyan
from torch.utils.tensorboard import SummaryWriter




In [3]:
setup = 'setup1'
dataType = '2D'
sampling = 'Single'
nClip = 1
maskType = 'None'
nw = 8

learning_rate = 1.0e-4

In [4]:
seed = 1337

# Set seed for PyTorch (CPU)
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

# Set seed for PyTorch (GPU)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # If you use multiple GPUs

# Make PyTorch operations deterministic (slows down computation but improves reproducibility)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')
print(torch.cuda.get_device_name())

test_name = 'default'
output_path = Path('/home/jonathan/sparse_recovery_tests/tests').joinpath(test_name)
output_path.mkdir(parents=True, exist_ok=True)

data_files_list = Path().joinpath('/data', 'datasets', 'N-UCLA_MA_3D')
if data_files_list.exists():
        data_files_list = data_files_list.as_posix() + f"/data/CV/{setup}/"

writer = SummaryWriter(output_path)


dyan = Dyan(device, 'dyan_config')
dyan.to(device)

# Print out parameters and their requires_grad status
# for name, param in dyan.named_parameters():
#     print(f"Parameter name: {name}, requires_grad={param.requires_grad}")

trainSet = NUCLA_CrossView(root_list=data_files_list, phase='train',
                            setup=setup, dataType=dataType,
                            sampling=sampling, nClip=nClip,
                            T=dyan.T, maskType=maskType) 
trainLoader = DataLoader(trainSet, shuffle=True,
                             batch_size=dyan.batch_size, num_workers=nw)

testSet = NUCLA_CrossView(root_list=data_files_list, phase='test',
                            setup=setup, dataType=dataType,
                            sampling=sampling, nClip=nClip,
                            T=dyan.T, maskType=maskType) 
testLoader = DataLoader(trainSet, shuffle=False,
                             batch_size=dyan.batch_size, num_workers=nw)

optimizer = torch.optim.SGD(filter(lambda x: x.requires_grad, dyan.parameters()), 
                            lr=learning_rate, weight_decay=0.001, momentum=0.9)

loss = nn.MSELoss()

for i, sample in enumerate(trainLoader):
    # setup batches
    skeletons = sample['input_skeletons']['normSkeleton'].float().to(device)
    t = skeletons.shape[1] # (batch_size x num_clips) x t x num_joint x dim_joint
    input_skeletons = skeletons.reshape(skeletons.shape[0], t, -1).to(device)
    
    optimizer.zero_grad()
    
    Y_pred, C_pred = dyan(input_skeletons)
    
    if i == 0:
        break



NVIDIA GeForce RTX 2080 Ti
Required key 'dictionary_filename' not found in configuration file. Using default value of None
Loaded configuration from /home/jonathan/sparse_recovery_tests/config/dyan_config.yaml
Generated random dictionary polesr cuda:0 th cuda:0
 and normalized it.


ValueError: optimizer got an empty parameter list

In [6]:
for name, param in dyan.named_parameters():
    print(f"Parameter: {name}, requires_grad: {param.requires_grad}")